# Quick Start
>Minimal Example of HierachicalForecast

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/TourismSmall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Large collections of time series organized into structures at different aggregation levels often require their forecasts to follow their aggregation constraints, which poses the challenge of creating novel algorithms capable of coherent forecasts. 

The `HierarchicalForecast` package provides a wide collection of Python implementations of hierarchical forecasting algorithms that follow classic hierarchical reconciliation.

Hierarchical Forecast or Hierarchical Reconciliation of Forecasts refers to the process of coherently reconciling forecasts of different levels or hierarchies. 

This guide assumes you have historical data at the lowest level of aggregation. 

You will:

1. Instal the necessary libraries

1. Do the necessary data wrangling.
In this step, you will start with historical data at the lowest level of aggregation. E.g. Sales at a product store level and you will generate a hierachical data frame for your past observations (Y_df_hierachical).

1. Use StatsForecast to forecast every possible level of your data.
In this step, you will use Y_hierachical_df as input for your forescasts and generate Y_hierachical_forecasts_df. 

1. Reconcile those forecast coherently
In this step, you will take Y_hierachical_forecasts_df and generate a reconciled version. 


## Install libraries

We assume you have HierachicalForecast already installed. Check this guide for instructions on [how to install HierachicalForecast](./Installation.ipynb).

Additionally, we will install `statsforecast` to generate blazing-fast forecasts. You can read more about StatsForecast [here](https://github.com/nixtla/statsforecast).

Install the necessary packages using `pip install statsforecast`
``

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast

In [ ]:
import numpy as np
import pandas as pd

#obtain hierarchical data
from datasetsforecast.hierarchical import HierarchicalData

# compute base forecast no coherent
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

#obtain hierarchical reconciliation methods and evaluation
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut

## Read the Data

In this example we will use the `TourismSmall` dataset. The following cell gets the time series for the different levels in the hierarchy, the summing matrix `S` which recovers the full dataset from the bottom level hierarchy and the indices of each hierarchy denoted by `tags`.

In [ ]:
Y_df, S_df, tags = HierarchicalData.load('./data', 'TourismSmall')
####### Kin, creo que hay que cambiar el ejemplo  a una bottom solamente. 

In [ ]:
Y_df.head()

,unique_id,ds,y
0,total,1998-03-31,84503
1,total,1998-06-30,65312
2,total,1998-09-30,72753
3,total,1998-12-31,70880
4,total,1999-03-31,86893


We split the dataframe in train/test splits.

## Split Data

In [ ]:
Y_test_df = Y_df.groupby('unique_id').tail(12)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_test_df = Y_test_df.set_index('unique_id')
Y_train_df = Y_train_df.set_index('unique_id')

## Generate Hierarchical Data Frame

Pendiente

## Forecast all levels 

The following cell computes the *base forecast* for each time series using the `auto_arima` and `naive` models. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

We fit the models by instantiating a new `StatsForecast` object with the following parameters:

* `models`: a list of models. Select the models you want from StatsForecast.

* `freq`: a string indicating the frequency of the data. (See [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).)

* `n_jobs`: n_jobs: int, number of jobs used in the parallel processing, use -1 for all cores.

* `fallback_model`: a model to be used if a model fails. 


Any settings are passed into the constructor. Then you call its fit method and pass in the historical data frame.


In [ ]:
%%capture
sf = StatsForecast(
    df=Y_train_df, 
    models=[AutoARIMA(season_length=12), Naive()], 
    freq='M', 
    n_jobs=-1
)
Y_hat_df = sf.forecast(h=12)

## Reconcile them

The following cell makes the previous forecasts coherent using the `HierarchicalReconciliation` class. The used methods to make the forecasts coherent are:
- `BottomUp`: The reconciliation of the method is a simple addition to the upper levels.
- `TopDown`: The second method constrains the base-level predictions to the top-most aggregate-level serie and then distributes it to the disaggregate series through the use of proportions. 
- `MiddleOut`: Anchors the base predictions in a middle level.

In [ ]:
reconcilers = [
    BottomUp(),
    TopDown(method='forecast_proportions'),
    MiddleOut(middle_level='Country/Purpose/State', 
              top_down_method='forecast_proportions')
]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
Y_rec_df = hrec.reconcile(Y_hat_df=Y_hat_df, Y_df=Y_train_df, 
                          S=S_df, tags=tags)

## Evaluate your coherent forecasts

The `HierarchicalForecast` package includes the `HierarchicalEvaluation` class to evaluate the different hierarchies and also is capable of compute scaled metrics compared to a benchmark model.

In [ ]:
def mse(y, y_hat):
    return np.mean((y-y_hat)**2)

evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluation = evaluator.evaluate(
        Y_hat_df=Y_rec_df, Y_test_df=Y_test_df, 
        tags=tags, benchmark='Naive'
)
evaluation.filter(like='ARIMA', axis=1).T

level,Overall,Country,Country/Purpose,Country/Purpose/State,Country/Purpose/State/CityNonCity
metric,mse-scaled,mse-scaled,mse-scaled,mse-scaled,mse-scaled
AutoARIMA,0.959048,1.051918,0.906252,0.909365,0.845243
AutoARIMA/BottomUp,0.914598,0.988955,0.869388,0.864788,0.845243
AutoARIMA/TopDown_method-forecast_proportions,0.972468,1.051918,0.88779,0.97276,0.9452
AutoARIMA/MiddleOut_middle_level-Country/Purpose/State_top_down_method-forecast_proportions,0.895918,0.950773,0.830191,0.909365,0.883235


### References
- [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American 
Economic Review, 58 , 773{787)](http://www.jstor.org/stable/1815532).
- [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. 
doi:10.1002/for.3980090304](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304).<br>
- [An investigation of aggregate variable time series forecast strategies with specific subaggregate 
time series statistical correlation. Computers and Operations Research, 26 , 1133–1149. 
doi:10.1016/S0305-0548(99)00017-9](https://doi.org/10.1016/S0305-0548(99)00017-9).
- [Hyndman, R.J., & Athanasopoulos, G. (2021). "Forecasting: principles and practice, 3rd edition: 
Chapter 11: Forecasting hierarchical and grouped series.". OTexts: Melbourne, Australia. OTexts.com/fpp3 
Accessed on July 2022.](https://otexts.com/fpp3/hierarchical.html)